# **Informe: Web Scraping en Datos Abiertos**

## Introducción

Web scraping es una técnica de extracción de datos estructurados desde sitios web de manera automatizada. Permite recolectar información publicada en línea y almacenarla en formatos manejables como CSV o bases de datos para su posterior análisis. En este informe, se documenta el uso de dos herramientas populares para realizar web scraping: _BeautifulSoup_ y _Selenium_, aplicado al sitio datosabiertos.gob.ec, el portal de datos abiertos del gobierno de Ecuador.

## Objetivos

- Comprender el funcionamiento de herramientas de web scraping en Python.

- Realizar pruebas con las librerías BeautifulSoup y Selenium.

- Extraer información de títulos y enlaces de datasets publicados en el sitio.

- Guardar los datos obtenidos en un archivo CSV para su análisis.

## Tecnologías y Herramientas

**Librerías Utilizadas**

1. _BeautifulSoup_: (parte de bs4): Para analizar el contenido HTML de la página web y extraer información estructurada.

2. _Selenium_: Para interactuar con páginas web que cargan contenido dinámico mediante JavaScript.

3. _Pandas_: Para manejar y guardar los datos en un formato CSV.

4. _Requests_: Para realizar solicitudes HTTP y obtener el contenido HTML de la página.

**Software**

- Python 3.12.4

- Navegador Google Chrome

- ChromeDriver

**Instalación de Librerías**

Antes de ejecutar los scripts, es necesario instalar las librerías requeridas mediante el siguiente comando:

    pip install requests beautifulsoup4 pandas selenium


## Desarrollo

### BeautifulSoup

**Procedimiento:**

1. Realizar una solicitud HTTP al sitio web [https://datosabiertos.gob.ec/](https://datosabiertos.gob.ec/) utilizando la librería requests.

2. Analizar el contenido HTML con BeautifulSoup.

3. Identificar los selectores CSS correspondientes a los títulos y enlaces de los datasets.

4. Extraer los datos y guardarlos en un archivo CSV.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL del sitio web
url = "https://datosabiertos.gob.ec/"

# Realizar la solicitud HTTP
response = requests.get(url)
response.raise_for_status()  # Asegura que no hubo errores en la solicitud

# Analizar el contenido HTML
soup = BeautifulSoup(response.text, "html.parser")

# Extraer los enlaces de los datasets
datasets = []
for item in soup.select("h3.dataset-title"):  # Cambiar el selector según sea necesario
    title = item.text.strip()
    link = item.find_parent("a").get("href")  # Encuentra el enlace contenedor del título
    if link and not link.startswith("http"):
        link = url + link  # Convertir enlaces relativos en absolutos
    datasets.append({"Título": title, "Enlace": link})

# Extraer los enlaces a los archivos CSS
css_links = []
for link in soup.find_all("link", rel="stylesheet"):  # Buscamos solo los enlaces de tipo CSS
    href = link.get("href")
    if href:
        if not href.startswith("http"):
            href = url + href  # Asumiendo que el enlace es relativo y necesitamos convertirlo
        css_links.append({"Archivo CSS": href})

# Combinar los datasets y los enlaces de los archivos CSS en un solo DataFrame
all_data = datasets + css_links

# Guardar todo en un archivo CSV
df = pd.DataFrame(all_data)
df.to_csv("datasets_datosabiertos_bs.csv", index=False, encoding="utf-8")
print("Datos guardados en 'datasets_datosabiertos_bs.csv'.")


Datos guardados en 'datasets_datosabiertos_bs.csv'.


### Selenium

**Procedimiento:**

1. Configurar Selenium con el controlador del navegador Chrome.

2. Cargar la página principal del sitio web.

3. Identificar los elementos que contienen los títulos y enlaces mediante selectores CSS.

4. Extraer los datos y guardarlos en un archivo CSV.

In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Abrir el sitio web
url = "https://datosabiertos.gob.ec/"
driver.get(url)
time.sleep(10)  # Esperar a que cargue el contenido

# Extraer títulos y enlaces
datasets = []
elements = driver.find_elements(By.CSS_SELECTOR, "body > link:nth-child(8)")

# Debug: Print the number of elements found
print(f"Number of elements found: {len(elements)}")

for element in elements:
    title = element.text.strip()
    link = element.get_attribute("href")
    datasets.append({"Título": title, "Enlace": link})

    # Debug: Print each title and link
    print(f"Title: {title}, Link: {link}")

# Guardar en un archivo CSV
df = pd.DataFrame(datasets)
df.to_csv("datasets_datosabiertos_selenium.csv", index=False, encoding="utf-8")

driver.quit()

Number of elements found: 1
Title: , Link: https://datosabiertos.gob.ec/webassets/vendor/0b01aef1_font-awesome.css


## Resultados

- **BeautifulSoup:** Se logró extraer los títulos y enlaces de los datasets disponibles en la página. Los datos se guardaron en el archivo  datasets_datosabiertos_bs.csv.

- **Selenium:** La interacción con la página permitió obtener los mismos datos y guardarlos en el archivo datasets_datosabiertos_selenium.csv.

## Conclusiones

- BeautifulSoup es ideal para sitios con contenido estático, ofreciendo simplicidad y rapidez.

- Selenium es necesario para interactuar con páginas que dependen de JavaScript, aunque es más lenta y compleja.

- Ambos métodos lograron extraer los datos requeridos y guardarlos exitosamente en archivos CSV.

- Se recomienda utilizar BeautifulSoup cuando sea posible, priorizando Selenium solo para casos específicos.

## Recomendaciones

- Verificar los términos de uso del sitio web antes de realizar scraping.

- Automatizar la ejecución periódica del código si se requiere mantener los datos actualizados.

- Optimizar el uso de Selenium para reducir el tiempo de espera y mejorar el rendimiento.

**GitHub:** git@github.com: alda244



[GitHub Métodos Númericos - Repositorio](https://github.com/alda244/MN_ActividadesExtra/blob/main/MN_AE12/PereiraAlicia_ActividadExtra12.ipynb)